## Image Classification

The basic 7 steps for building models in general are listed so:

1. Load Dataset
2. Make Dataset Iterable
3. Create Model Class
4. Instantiate Model Class
5. Instantiate Loss Class
6. Instantiate Optimizer Class
7. Train Model

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
import torchvision.datasets as dsets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image


from torch.utils.data import Dataset, DataLoader

In [2]:
all_df = pd.read_table('list_eval_partition.txt', delim_whitespace=True)
labels_df = pd.read_table('list_category_img.txt', delim_whitespace=True)


/home/paperspace/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.
/home/paperspace/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  


In [3]:
def func(image_name):
    category_label = labels_df[labels_df['image_name'].str.match(image_name)].iloc[0]['category_label']
    create_new = np.zeros(50)
    create_new[category_label - 1] = 1
    return create_new

In [4]:
x = labels_df["category_label"].value_counts()
# x.index.sort_values()
x

41    72158
18    36887
3     24557
32    19666
17    15429
33    14773
6     13311
16    13123
11    10467
19    10078
2      7495
48     7408
26     7076
42     6153
30     5013
29     4416
10     4048
34     3048
44     2294
39     2120
24     1669
35     1106
15      791
12      748
9       716
13      676
27      594
22      527
23      486
36      386
5       330
7       324
4       309
1       160
47      150
20      146
43      126
14       97
21       77
46       70
37       54
25       49
28       45
31       32
8        17
40       17
Name: category_label, dtype: int64

In [5]:
useless_attributes = [38,45,49,50]
low_count_attr = [1,4,5,7,8,14,20,21,22,23,27,13,9,12,15,25,28,31,36,46,37,40,43,46,47]

In [6]:
for low in low_count_attr:
    labels_df = labels_df[labels_df['category_label'] != low]
labels_df

,image_name,category_label
0,img/Sheer_Pleated-Front_Blouse/img_00000001.jpg,3
1,img/Sheer_Pleated-Front_Blouse/img_00000002.jpg,3
2,img/Sheer_Pleated-Front_Blouse/img_00000003.jpg,3
3,img/Sheer_Pleated-Front_Blouse/img_00000004.jpg,3
4,img/Sheer_Pleated-Front_Blouse/img_00000005.jpg,3
5,img/Sheer_Pleated-Front_Blouse/img_00000006.jpg,3
6,img/Sheer_Pleated-Front_Blouse/img_00000007.jpg,3
7,img/Sheer_Pleated-Front_Blouse/img_00000008.jpg,3
8,img/Sheer_Pleated-Front_Blouse/img_00000009.jpg,3
9,img/Sheer_Pleated-Front_Blouse/img_00000010.jpg,3


In [7]:
lower = [41,18,3,32,17,33,6,16,11,19]

for elem in lower:  
    jon = labels_df[labels_df['category_label'] == elem]
    drop_diff = len(labels_df[labels_df['category_label'] == elem]) - 10000
    # random_vector = np.random.choice(drop_diff + 10000, drop_diff)
    # labels_df = labels_df.drop(labels_df.[random_vector])
    # labels_df
    df_dropped = jon.drop(jon.sample(n=drop_diff).index)
    labels_df = labels_df[labels_df['category_label'] != elem]
    labels_df = labels_df.append(df_dropped)
labels_df

# duplicate = [35,24,39,44,34,10,29,30,42,26,48,2]

# for elem in duplicate:
#     snow = labels_df[labels_df['category_label'] == elem]
#     sample_diff = 10000 - len(snow)
#     df_dropped = snow.sample(n=10000, replace=True)
#     labels_df = labels_df[labels_df['category_label'] != elem]
#     labels_df = labels_df.append(df_dropped)

# indices = np.arange(len(labels_df))
# labels_df.set_index(indices)

,image_name,category_label
750,img/Single-Button_Blazer/img_00000001.jpg,2
751,img/Single-Button_Blazer/img_00000002.jpg,2
752,img/Single-Button_Blazer/img_00000003.jpg,2
753,img/Single-Button_Blazer/img_00000004.jpg,2
754,img/Single-Button_Blazer/img_00000005.jpg,2
755,img/Single-Button_Blazer/img_00000006.jpg,2
756,img/Single-Button_Blazer/img_00000007.jpg,2
757,img/Single-Button_Blazer/img_00000008.jpg,2
758,img/Single-Button_Blazer/img_00000009.jpg,2
759,img/Single-Button_Blazer/img_00000010.jpg,2


In [8]:
all_df = all_df[all_df.isin(labels_df)['image_name'] == True]
# print(len(all_df))
# all_df = labels_df[['image_name']].merge(all_df)
# all_df

In [9]:
'''
STEP 1: LOAD DATASET
'''
# test_df = pd.read_csv('fashionmnist/fashion-mnist_test.csv')
# test_df_labels = test_df['label']
# test_pixels_df = test_df.drop('label', axis=1)
validation_df = all_df[all_df['evaluation_status'].str.contains('val')]
# np.random.shuffle(validation_df)
validation_df = validation_df.drop(['evaluation_status'], axis=1)

train_df = all_df[all_df['evaluation_status'].str.contains('train')]
# np.random.shuffle(train_df)
train_df = train_df.drop(['evaluation_status'], axis=1)

test_df = all_df[all_df['evaluation_status'].str.contains('test')]
# np.random.shuffle(test_df)
test_df = test_df.drop(['evaluation_status'], axis=1)

# train_df = pd.read_csv('fashionmnist/fashion-mnist_train.csv')
# train_pixels_df = train_df.drop('label', axis=1)
# train_df_labels = train_df['label']
train_df

,image_name
0,img/Sheer_Pleated-Front_Blouse/img_00000001.jpg
3,img/Sheer_Pleated-Front_Blouse/img_00000004.jpg
7,img/Sheer_Pleated-Front_Blouse/img_00000008.jpg
8,img/Sheer_Pleated-Front_Blouse/img_00000009.jpg
10,img/Sheer_Pleated-Front_Blouse/img_00000011.jpg
13,img/Sheer_Pleated-Front_Blouse/img_00000014.jpg
15,img/Sheer_Pleated-Front_Blouse/img_00000016.jpg
17,img/Sheer_Pleated-Front_Blouse/img_00000018.jpg
20,img/Sheer_Pleated-Front_Blouse/img_00000021.jpg
23,img/Sheer_Pleated-Front_Blouse/img_00000024.jpg


In [10]:
# #np.unique(np.argmax(labelvalidation, axis=1), return_counts=True)

# validation_df.iloc[1334]['image_name']
# func('img/Open-Back_Knit_Blouse/img_00000020.jpg')

In [11]:
# np.unique(np.argmax(labeltrain, axis=1), return_counts=True)

In [12]:
allimagesvalidation = []
labelvalidation = []
# fiftyimagesArray = np.zeros(50)
for index in range(50):
#     create random number
    x = np.random.choice(20969)
    img_filepath = validation_df.iloc[x]['image_name']
    im = Image.open(img_filepath)
#     print(x)
    labelvalidation.append(func(img_filepath))
    imarr = np.uint8(np.asarray(im.convert('RGB').resize((224,224))))
    imarr = (imarr - imarr.mean())/imarr.std()
    allimagesvalidation.append(imarr)

allimagestrain = []
labeltrain = []
for index in range(500):
    x = np.random.choice(109826)
    img_filepath = train_df.iloc[x]['image_name']
    im = Image.open(img_filepath)
    labeltrain.append(func(img_filepath))
    imarr = np.uint8(np.asarray(im.convert('RGB').resize((224,224))))
    imarr = (imarr - imarr.mean())/imarr.std()
    allimagestrain.append(imarr)

# allimagestest = []
# labeltest = []
# for index in range(10000):
#     img_filepath = test_df.iloc[index]['image_name']
#     im = Image.open(img_filepath)
#     labeltest.append(func(img_filepath))
#     imarr = np.uint8(np.asarray(im.convert('RGB').resize((224,224))))
#     #imarr = np.round((imarr - imarr.mean())/imarr.std())
#     allimagestest.append(imarr)
# im = Image.open('414m1dOolTL._SX342_.jpg')
# imarr = np.uint8(np.asarray(im.convert('RGB').resize((224,224))))

# fiftyimagesArray

In [13]:
# '''
# STEP 1.5: defining and instantiating Dataset subclass 
# '''

# '''
# This is our custom Dataset class. Remember from 1st meeting that we need this to pipeline our data into training our model.

# The pipeline is important!!! At larger scale, machine learning can get bottlenecked at disk reads (in image classification for example)
# so understanding the various stages is important. We don't have to worry about that kind of stuff now since we're just creating small
# project models as opposed to complex production models.

# NOTE: this is not the only way to create a dataset. An alternative is to simply pass in a dataframe that contains both pixel and label data.
# Then we can index the label and pixel data inside of __getitem__ as opposed to separating labels and pixel data before hand like I did.
# '''
# class FashionDataset(Dataset):
#     def __init__(self, dataframe, labels):
#         self.labels = torch.LongTensor(labels)
#         self.df = dataframe
        
#     def __getitem__(self, index):
#         # I'm using .loc to access the row of the dataframe by index
#         a = self.df.loc[index]
# #         a = (a - np.mean(a))/np.std(a)
#         img = torch.Tensor(a)
#         label = self.labels[index]
#         return img, label

#     def __len__(self):
#         return len(self.labels)
    
# '''
# This class is for providing image data as (1, 28, 28) tensor as opposed to a (784) tensor. You
# use these for conv2d layers which are powerful for image recognition!

# NOTE: Please note that I normalized the data VERY INCORRECTLY. Here I am normalizing the data across 
# each sample individually which is not good. I should be normalizing across the ENTIRE training data set.

# Also, when I create the test dataset I should normalize it based on the TRAINING set's mean and standard deviation.
# Since the model is trained on the training data, we want to make sure that we transform the test data the same way we
# transform the training data. Otherwise it's like training a model to do one job and then testing it by on another job.
# '''
# class Fashion2DDataset(Dataset):
#     def __init__(self, dataframe, labels):
#         self.labels = torch.LongTensor(labels)
#         self.df = dataframe
        
#     def __getitem__(self, index):
#         # I'm using .loc to access the row of the dataframe by index
#         a = self.df.loc[index]
#         a = (a - np.mean(a))/np.std(a)
#         a = np.split(a, 28)
#         a = np.array([a])
#         img = torch.Tensor(a)
        
#         label = self.labels[index]
#         return img, label

#     def __len__(self):
#         return len(self.labels)

class ClothingDataset(Dataset):
    def __init__(self, data, labels):
        self.labels = labels
        self.data = data
        
    def __getitem__(self, index):
        img = self.data[index]
        
        trans = transforms.ToTensor()
        img = trans(img).float()
        
        label = self.labels[index]
        label = torch.LongTensor(label)
        
        return img, label

    def __len__(self):
        return len(self.labels)

In [14]:
'''
STEP 2: MAKING DATASET ITERABLE
'''
# train_dataset = Fashion2DDataset(train_pixels_df, train_df_labels.values)
# test_dataset = Fashion2DDataset(test_pixels_df, test_df_labels.values)

# batch_size = 100

# train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
#                                            batch_size=batch_size, 
#                                            shuffle=True)

# test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
#                                           batch_size=batch_size, 
#  shuffle=False)

batch_size = 25

train_dataset = ClothingDataset(allimagestrain, labeltrain)
validation_dataset = ClothingDataset(allimagesvalidation, labelvalidation) 
# test_dataset = ClothingDataset(allimagestest, labeltest)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size,                                             
                                           shuffle=True)
# test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
#                                           batch_size=batch_size,
#                                           shuffle=True)
validation_loader = torch.utils.data.DataLoader(dataset=validation_dataset, 
                                          batch_size=batch_size,
                                          shuffle=True)




In [15]:
'''
STEP 3: CREATE MODEL CLASS
'''
# class FeedforwardNeuralNetModel(nn.Module):
#     def __init__(self, input_dim, hidden_dim, output_dim):
#         super(FeedforwardNeuralNetModel, self).__init__()
#         self.fc1 = nn.Linear(input_dim, hidden_dim) 
#         self.relu = nn.ReLU()
#         self.fc2 = nn.Linear(hidden_dim, output_dim)  

#     def forward(self, x):
#         out = self.fc1(x)
#         out = self.relu(out)
#         out = self.fc2(out)
#         return out
    
class ConvolutionalNeuralNetModel(nn.Module):
    def __init__(self, output_dim):
        super(ConvolutionalNeuralNetModel, self).__init__()
#         self.conv1 = nn.Conv2d(3, 10, kernel_size=56, stride=2, padding=2, bias=False)
#         self.bn1 = nn.BatchNorm2d(10)
#         self.relu = nn.ReLU(inplace=True)
#         self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
#         self.fc = nn.Linear(19360, output_dim)
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 25, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(25, 3, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(25, 3, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(9408, 9408)
        self.fc2 = nn.Linear(9408, 9408)
        self.fc3 = nn.Linear(out)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

In [16]:
'''
STEP 4: INSTANTIATE MODEL CLASS
'''
#input_dim = 224*224
#hidden_dim = 5000
output_dim = 50 

# model = FeedforwardNeuralNetModel(input_dim, hidden_dim, output_dim)
#model = ConvolutionalNeuralNetModel(output_dim)
model = models.resnet18(pretrained=True)

In [17]:
'''
STEP 5: INSTANTIATE LOSS CLASS
'''
criterion = nn.CrossEntropyLoss()

In [18]:

'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''
"""
Most of the time I use SGD. Feel free to use another optimizer if you wish.
What hyperparameters would you use/set here?
"""
learning_rate = .1

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [21]:
'''
STEP 7: TRAIN THE MODEL
'''
iter = 0
losses = []
accuracies = []
for epoch in range(100):
    for i, (images, labels) in enumerate(train_loader):
        # Load images with gradient accumulation capabilities
#         images = images.view(-1, 28*28).requires_grad_()

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        outputs = model(images)
        labels = torch.max(labels,1)[1]
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 1 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in validation_loader:
                # Load images with gradient accumulation capabilities
                # images = images.view(-1, 28*28).requires_grad_()

                # Forward pass only to get logits/output
                outputs = model(images)

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                labels = torch.max(labels,1)[1]

                # Total number of labels
                total += labels.size(0)

                # Total correct predictions
                correct += (predicted == labels).sum()

            accuracy =  100 * correct / total
            
            accuracies.append(accuracy)
            losses.append(loss.item())
            
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 1. Loss: 0.0012039184803143144. Accuracy: 18
Iteration: 2. Loss: 0.00037971496931277215. Accuracy: 22
Iteration: 3. Loss: 0.0004096984921488911. Accuracy: 20
Iteration: 4. Loss: 0.0002816772321239114. Accuracy: 24
Iteration: 5. Loss: 0.00034378052805550396. Accuracy: 20
Iteration: 6. Loss: 0.0005784606910310686. Accuracy: 22
Iteration: 7. Loss: 0.0006861114525236189. Accuracy: 20
Iteration: 8. Loss: 0.0012207793770357966. Accuracy: 22
Iteration: 9. Loss: 0.00040725708822719753. Accuracy: 22
Iteration: 10. Loss: 0.0011867523426190019. Accuracy: 24
Iteration: 11. Loss: 0.007437057327479124. Accuracy: 22
Iteration: 12. Loss: 0.008965606801211834. Accuracy: 26
Iteration: 13. Loss: 0.0004527282726485282. Accuracy: 20
Iteration: 14. Loss: 0.0006258392240852118. Accuracy: 18
Iteration: 15. Loss: 0.0018978881416842341. Accuracy: 22
Iteration: 16. Loss: 0.0002571868826635182. Accuracy: 24
Iteration: 17. Loss: 0.0005566406180150807. Accuracy: 14
Iteration: 18. Loss: 0.0009436798281967

Iteration: 145. Loss: 0.0005702209309674799. Accuracy: 20
Iteration: 146. Loss: 0.0005661773611791432. Accuracy: 14
Iteration: 147. Loss: 0.0004812622210010886. Accuracy: 22
Iteration: 148. Loss: 0.0005778503254987299. Accuracy: 26
Iteration: 149. Loss: 0.0009644317906349897. Accuracy: 22
Iteration: 150. Loss: 0.0008400726364925504. Accuracy: 20
Iteration: 151. Loss: 0.0006111908005550504. Accuracy: 20
Iteration: 152. Loss: 0.0005121612339280546. Accuracy: 20
Iteration: 153. Loss: 0.0005804443499073386. Accuracy: 22
Iteration: 154. Loss: 0.00034332275390625. Accuracy: 16
Iteration: 155. Loss: 0.0006031799130141735. Accuracy: 22
Iteration: 156. Loss: 0.0007762908935546875. Accuracy: 18
Iteration: 157. Loss: 0.0001653289800742641. Accuracy: 22
Iteration: 158. Loss: 0.0003330230829305947. Accuracy: 22
Iteration: 159. Loss: 0.0016596984351053834. Accuracy: 24
Iteration: 160. Loss: 0.0005355834728106856. Accuracy: 20
Iteration: 161. Loss: 0.0008206939673982561. Accuracy: 20
Iteration: 162. 

Iteration: 287. Loss: 0.0004665374872274697. Accuracy: 22
Iteration: 288. Loss: 0.00013473510625772178. Accuracy: 24
Iteration: 289. Loss: 0.00035072327591478825. Accuracy: 20
Iteration: 290. Loss: 0.0006639098864980042. Accuracy: 20
Iteration: 291. Loss: 0.0003100585890933871. Accuracy: 20
Iteration: 292. Loss: 0.0004226684686727822. Accuracy: 22
Iteration: 293. Loss: 0.0019891357515007257. Accuracy: 20
Iteration: 294. Loss: 0.00010559082147665322. Accuracy: 20
Iteration: 295. Loss: 0.0009846496395766735. Accuracy: 22
Iteration: 296. Loss: 0.00029800416086800396. Accuracy: 20
Iteration: 297. Loss: 0.0006199646159075201. Accuracy: 20
Iteration: 298. Loss: 0.0002539825509302318. Accuracy: 20
Iteration: 299. Loss: 0.0005614471156150103. Accuracy: 26
Iteration: 300. Loss: 0.00030799864907748997. Accuracy: 18
Iteration: 301. Loss: 0.0002988433698192239. Accuracy: 22
Iteration: 302. Loss: 0.00028900147299282253. Accuracy: 24
Iteration: 303. Loss: 0.00029853821615688503. Accuracy: 22
Iterati

Iteration: 428. Loss: 0.0002606963971629739. Accuracy: 22
Iteration: 429. Loss: 0.00013648986350744963. Accuracy: 20
Iteration: 430. Loss: 0.0005401611560955644. Accuracy: 22
Iteration: 431. Loss: 0.0029719544108957052. Accuracy: 20
Iteration: 432. Loss: 0.00042297362233512104. Accuracy: 22
Iteration: 433. Loss: 0.0028584289830178022. Accuracy: 22
Iteration: 434. Loss: 0.0006199646159075201. Accuracy: 22
Iteration: 435. Loss: 0.0003814697265625. Accuracy: 20
Iteration: 436. Loss: 0.00015136718866415322. Accuracy: 22
Iteration: 437. Loss: 0.0008784484816715121. Accuracy: 22
Iteration: 438. Loss: 0.0009053039830178022. Accuracy: 24
Iteration: 439. Loss: 0.00030517578125. Accuracy: 22
Iteration: 440. Loss: 0.00041366578079760075. Accuracy: 24
Iteration: 441. Loss: 0.0002011871401919052. Accuracy: 24
Iteration: 442. Loss: 0.0001979827939067036. Accuracy: 24
Iteration: 443. Loss: 0.0005323028308339417. Accuracy: 24
Iteration: 444. Loss: 0.00032661439036019146. Accuracy: 22
Iteration: 445. L

Iteration: 569. Loss: 0.0004822540213353932. Accuracy: 22
Iteration: 570. Loss: 0.0003722381661646068. Accuracy: 20
Iteration: 571. Loss: 8.96453857421875e-05. Accuracy: 24
Iteration: 572. Loss: 0.00027603149646893144. Accuracy: 22
Iteration: 573. Loss: 0.00012825011799577624. Accuracy: 22
Iteration: 574. Loss: 0.00013847350783180445. Accuracy: 22
Iteration: 575. Loss: 0.00035873413435183465. Accuracy: 16
Iteration: 576. Loss: 0.0001090240475605242. Accuracy: 24
Iteration: 577. Loss: 0.00015747069846838713. Accuracy: 24
Iteration: 578. Loss: 0.00014564514276571572. Accuracy: 24
Iteration: 579. Loss: 0.0008174896356649697. Accuracy: 22
Iteration: 580. Loss: 0.00010383605695096776. Accuracy: 22
Iteration: 581. Loss: 0.0004273223748896271. Accuracy: 20
Iteration: 582. Loss: 0.00026908874860964715. Accuracy: 22
Iteration: 583. Loss: 0.0003147888055536896. Accuracy: 22
Iteration: 584. Loss: 0.00030082702869549394. Accuracy: 22
Iteration: 585. Loss: 0.00017333983851131052. Accuracy: 22
Itera

...

Iteration: 1267. Loss: 0.00019920349586755037. Accuracy: 26
Iteration: 1268. Loss: 0.00047584532876498997. Accuracy: 26
Iteration: 1269. Loss: 0.00012153625721111894. Accuracy: 24
Iteration: 1270. Loss: 8.834838808979839e-05. Accuracy: 22
Iteration: 1271. Loss: 4.394531424622983e-05. Accuracy: 24
Iteration: 1272. Loss: 0.00015380859258584678. Accuracy: 22
Iteration: 1273. Loss: 7.873534923419356e-05. Accuracy: 22
Iteration: 1274. Loss: 8.926391456043348e-05. Accuracy: 24
Iteration: 1275. Loss: 0.00044387817615643144. Accuracy: 24
Iteration: 1276. Loss: 7.324219041038305e-05. Accuracy: 26
Iteration: 1277. Loss: 6.67572021484375e-05. Accuracy: 26
Iteration: 1278. Loss: 0.0002317809994565323. Accuracy: 24
Iteration: 1279. Loss: 8.850097947288305e-05. Accuracy: 20
Iteration: 1280. Loss: 0.0001338195870630443. Accuracy: 24
Iteration: 1281. Loss: 0.0002483367861714214. Accuracy: 22
Iteration: 1282. Loss: 0.0002575683465693146. Accuracy: 24
Iteration: 1283. Loss: 0.004138412419706583. Accurac

Iteration: 1405. Loss: 0.00014930724864825606. Accuracy: 20
Iteration: 1406. Loss: 0.0003436279366724193. Accuracy: 24
Iteration: 1407. Loss: 0.0001776886056177318. Accuracy: 24
Iteration: 1408. Loss: 0.0001882171636680141. Accuracy: 22
Iteration: 1409. Loss: 0.00027328490978106856. Accuracy: 22
Iteration: 1410. Loss: 0.0001261901925317943. Accuracy: 20
Iteration: 1411. Loss: 0.00013496399333234876. Accuracy: 22
Iteration: 1412. Loss: 0.00033241271739825606. Accuracy: 22
Iteration: 1413. Loss: 0.0002062988351099193. Accuracy: 18
Iteration: 1414. Loss: 0.0002781677176244557. Accuracy: 24
Iteration: 1415. Loss: 9.544372733216733e-05. Accuracy: 24
Iteration: 1416. Loss: 8.537292160326615e-05. Accuracy: 24
Iteration: 1417. Loss: 0.00034698485978879035. Accuracy: 22
Iteration: 1418. Loss: 0.00011329651169944555. Accuracy: 26
Iteration: 1419. Loss: 0.00017845153342932463. Accuracy: 22
Iteration: 1420. Loss: 4.074096796102822e-05. Accuracy: 20
Iteration: 1421. Loss: 0.0002711486886255443. Acc

Iteration: 1544. Loss: 0.0004774475237354636. Accuracy: 22
Iteration: 1545. Loss: 0.0001405334478477016. Accuracy: 24
Iteration: 1546. Loss: 0.0002161407464882359. Accuracy: 22
Iteration: 1547. Loss: 5.6381224567303434e-05. Accuracy: 24
Iteration: 1548. Loss: 7.423400529660285e-05. Accuracy: 22
Iteration: 1549. Loss: 0.00018585205543786287. Accuracy: 26
Iteration: 1550. Loss: 0.00024391173792537302. Accuracy: 22
Iteration: 1551. Loss: 9.078979201149195e-05. Accuracy: 22
Iteration: 1552. Loss: 5.798339770990424e-05. Accuracy: 22
Iteration: 1553. Loss: 5.645752025884576e-05. Accuracy: 24
Iteration: 1554. Loss: 0.00020599365234375. Accuracy: 24
Iteration: 1555. Loss: 0.0001230621273862198. Accuracy: 22
Iteration: 1556. Loss: 0.0002963256847579032. Accuracy: 24
Iteration: 1557. Loss: 0.00018051147344522178. Accuracy: 24
Iteration: 1558. Loss: 0.00016998291539493948. Accuracy: 26
Iteration: 1559. Loss: 0.00012290955055505037. Accuracy: 22
Iteration: 1560. Loss: 0.0001738739083521068. Accura

Iteration: 1683. Loss: 0.00019309997151140124. Accuracy: 22
Iteration: 1684. Loss: 0.0001497650082455948. Accuracy: 24
Iteration: 1685. Loss: 4.806518700206652e-05. Accuracy: 22
Iteration: 1686. Loss: 3.5400389606365934e-05. Accuracy: 18
Iteration: 1687. Loss: 7.339477451751009e-05. Accuracy: 20
Iteration: 1688. Loss: 0.0002830505254678428. Accuracy: 22
Iteration: 1689. Loss: 9.811401105253026e-05. Accuracy: 26
Iteration: 1690. Loss: 0.00014564514276571572. Accuracy: 20
Iteration: 1691. Loss: 0.0004666137683670968. Accuracy: 24
Iteration: 1692. Loss: 0.000179290771484375. Accuracy: 24
Iteration: 1693. Loss: 0.00011047362932004035. Accuracy: 24
Iteration: 1694. Loss: 0.0001277923583984375. Accuracy: 22
Iteration: 1695. Loss: 0.0001942443777807057. Accuracy: 22
Iteration: 1696. Loss: 0.00021759033552370965. Accuracy: 20
Iteration: 1697. Loss: 0.0001459503109799698. Accuracy: 22
Iteration: 1698. Loss: 3.7841797166038305e-05. Accuracy: 22
Iteration: 1699. Loss: 0.0001468658447265625. Accur

KeyboardInterrupt: 

In [22]:
torch.save(model.state_dict(), 'resnet18proto')